### Use Chat GPT to engineer sentiment on Reddit data

In [9]:
# load api key
# load library to get environmental files
import os
from dotenv import load_dotenv


# load keys from  environmental var
load_dotenv() # .env file in cwd
gpt_key = os.environ.get("gpt_key") 

In [11]:
import requests 
import pandas as pd
import numpy as np

In [13]:
## function to query a response from Chat GPT
def ask_chat(question, api_key):
    # define headers
    headers = {
        "Authorization": f"Bearer {gpt_key}",
        "Content-Type": "application/json",
    }

    info = {
     "model": "gpt-4o-mini",
     "messages": [{"role": "user", "content": question}],
     "temperature": 0
    }
    ##Get the response
    response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, json = info)
    ##Convert to json
    response_json = response.json()
    return response_json['choices'][0]['message']['content'].strip()
    

In [15]:
##open coded csv file
test_data = pd.read_csv("./data/reddit/coded_comment_test_set_12_6.csv")
test_data.head()

,Unnamed: 0.1,Unnamed: 0,comment_id,submission_id,author,body,score,year,month,safety_code
0,0,0,isl1dig,y57cbw,eeek0711,Takoma Park MD,3,2022,10,0
1,1,1,i9cnmct,utr8aw,dans_cafe,"Generally speaking, robberies and muggings are...",1,2022,5,1
2,2,2,i9csov5,utr8aw,Ok_Priority_1534,Downtown/Central DC.\n\nAs I'm looking at diff...,1,2022,5,1
3,3,3,k7632z8,17ed6rg,Musictravels23,Hey there!! I have a fuIIy furnished sunny spa...,1,2023,10,0
4,4,4,io5bcjq,xciciq,cheesyuser,Thats a difficult ask. The 2000 range may be d...,2,2022,9,0


In [33]:
import time
##loop through the coded data to test gpt's accuracy with prompt
answer = []
for j in range(len(test_data)):
    try:
        base_question = "Is this text about neighborhood safety in DC? \
        Answer only with a number: 1 if about safety in DC, 0 if not. \
        Here is the text: "
        text = test_data.loc[j, "body"]
        full_question = base_question + str(text)
        answer.append(ask_chat(full_question, gpt_key))
        time.sleep(1)
    except:
        output.append(np.nan)
    

In [35]:
answer[2]

'1'

In [37]:
test_data['gpt_score'] = pd.to_numeric(answer)

In [39]:
test_data.head()

,Unnamed: 0.1,Unnamed: 0,comment_id,submission_id,author,body,score,year,month,safety_code,gpt_score
0,0,0,isl1dig,y57cbw,eeek0711,Takoma Park MD,3,2022,10,0,0
1,1,1,i9cnmct,utr8aw,dans_cafe,"Generally speaking, robberies and muggings are...",1,2022,5,1,1
2,2,2,i9csov5,utr8aw,Ok_Priority_1534,Downtown/Central DC.\n\nAs I'm looking at diff...,1,2022,5,1,1
3,3,3,k7632z8,17ed6rg,Musictravels23,Hey there!! I have a fuIIy furnished sunny spa...,1,2023,10,0,0
4,4,4,io5bcjq,xciciq,cheesyuser,Thats a difficult ask. The 2000 range may be d...,2,2022,9,0,0


In [41]:
test_data_acc = test_data.dropna().copy()

In [43]:
from sklearn.metrics import accuracy_score
# check accuracy
accuracy = accuracy_score(test_data_acc['safety_code'], test_data_acc['gpt_score'])
# see
print(accuracy)

0.825


In [45]:
test_data_acc.to_csv("./data/reddit/test_data_accuracy.csv")